In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scipy.stats as stats

### Import Features

In [2]:
filename = 'FDs_combined_03_03_22_analysis.csv'
# filename = 'relevant_features/calculated_relevantFeatures_2_standardized.csv'

calculated_features = pd.read_csv(filename)
calculated_features

,SID,lobe,artery_or_vein,label,num_files,files,FDs__Fractal_Dimension,FDs__R_Squared,FDs_20Size__Fractal_Dimension,FDs_20Size__R_Squared,...,FDs_20Size_allOffsets__FD@root1,FDs_20Size_allOffsets__FD@root2,FDs_DNN_SCALE_20SizeBy1_ArtVein__Fractal_Dimension,FDs_DNN_SCALE_20SizeBy1_ArtVein__R_Squared,FDs_DNN_SCALE_ArtVein__Fractal_Dimension,FDs_DNN_SCALE_ArtVein__R_Squared,FDs_20SizeBy1_allOffsets_ArtVein__Fractal_Dimension,FDs_20SizeBy1_allOffsets_ArtVein__R_Squared,FDs_20Size_allOffsets_ArtVein__Fractal_Dimension,FDs_20Size_allOffsets_ArtVein__R_Squared
0,62,left,NaN,control,5,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,2.386236,0.999772,2.371542,0.999145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62,left,NaN,control,0,[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62,left,artery,control,5,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62,left,artery,control,0,[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62,left,vein,control,5,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,298,whole,NaN,epah,2,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,2.193272,2.582255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1634,298,whole,artery,epah,7,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,NaN,NaN,1.739779,0.998255,2.028006,0.996622,NaN,NaN,NaN,NaN
1635,298,whole,artery,epah,2,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.074771,0.999301,2.083861,0.999371
1636,298,whole,vein,epah,7,['C:\\Users\\andre\\Documents\\GitHub\\sampleN...,NaN,NaN,NaN,NaN,...,NaN,NaN,1.658877,0.998576,1.928956,0.996985,NaN,NaN,NaN,NaN


### Functions: Generate Statistics

In [3]:
def generate_nick_stats(values_df, features, save=True, filename='test.csv'):
    df = values_df[features].describe().transpose()
    df.columns = [f'total__{col}' for col in df.columns.values]
    df.index.names = ['feature']
    
    features_list = []
    for feature in features:
        # Get basic statistics for each group
        temp_df = values_df[[feature, 'label']].groupby('label').describe().stack().T
        temp_df.columns = ['__'.join(col).strip() for col in temp_df.columns.values]
        temp_df.index.names= ['feature']
        
        features_dict = temp_df.to_dict(orient='records')[0]
        
        features_dict['feature'] = feature
        
        # Skip if data are not well behaved
        skip_msg = ''
        if len(values_df[feature].unique()) < 3:
            skip_msg = 'Not enough unique values! '
        elif values_df[feature].std() < 1e-12:
            skip_msg = 'Not enough variation! '

        features_dict['test_notes'] = skip_msg
        if len(skip_msg) > 1:
            features_list.append(features_dict)
            continue
        
        groups = [['control', 'epah'], ['control', 'pah'], ['epah', 'pah']]

        for group in groups:
            X1 = values_df[values_df['label'].eq(group[0])][feature].dropna()
            X2 = values_df[values_df['label'].eq(group[1])][feature].dropna()
            _, pvalue = stats.ttest_ind(X1, X2)
            features_dict[f't-test_{group[0]}_{group[1]}'] = pvalue

            _, pvalue = stats.mannwhitneyu(X1, X2)
            features_dict[f'wilcoxon_{group[0]}_{group[1]}'] = pvalue
        
        features_list.append(features_dict)
        
    df = df.merge(pd.DataFrame(features_list), on='feature')
    
    if save:
        df.to_csv(filename, index=False)
    
    return df

### Generate Nick Stats

In [5]:
# t-test + wilcoxon by pairs
filename = 'FDs_combined_03_03_22_analysis.csv'

calculated_features = pd.read_csv(filename)
calculated_features['artery_or_vein'].fillna('', inplace=True)
features = [col for col in calculated_features.columns if 'FDs' in col]

lobe_list = ['left', 'right', 'whole']
artvein_list = ['', 'artery', 'vein']

df_list = []
for lobe in lobe_list:
    for artvein in artvein_list:
        mask = calculated_features['lobe'].eq(lobe) & calculated_features['artery_or_vein'].eq(artvein)
        if not artvein:
            features = [col for col in calculated_features.columns if 'FDs' in col and 'ArtVein' not in col]
        else:
            features = [col for col in calculated_features.columns if 'FDs' in col and 'ArtVein' in col]
        df = generate_nick_stats(calculated_features[mask], features, save=False, filename=f'{lobe}_{artvein}_statistics.csv')
        df['lobe'] = lobe
        df['artery_or_vein'] = artvein
        df_list.append(df)

df_combined = pd.concat(df_list)
col_list = df_combined.columns.tolist()
df_combined = df_combined[['feature', 'lobe', 'artery_or_vein'] + col_list[1:-2]]
df_combined.to_csv('test.csv', index=False)

In [6]:
groups = [['control', 'epah'], ['control', 'pah'], ['epah', 'pah']]
feature = features[0]

print(f'Feature: {feature}\n')

for group in groups:
    mask = calculated_features['lobe'].eq(lobe) & calculated_features['artery_or_vein'].eq(artvein)
    X1 = calculated_features[calculated_features['label'].eq(group[0]) & mask][feature]
    X2 = calculated_features[calculated_features['label'].eq(group[1]) & mask][feature]
    _, pvalue = stats.ttest_ind(X1, X2)
    print(f't-test_{group[0]}_{group[1]}:    {pvalue}')
    
    _, pvalue = stats.mannwhitneyu(X1, X2)
    print(f'wilcoxon_{group[0]}_{group[1]}:  {pvalue}')
    print()

Feature: FDs_20SizeBy1_ArtVein__Fractal_Dimension

t-test_control_epah:    nan
wilcoxon_control_epah:  0.05225653096780727

t-test_control_pah:    nan
wilcoxon_control_pah:  0.07934847998328132

t-test_epah_pah:    nan
wilcoxon_epah_pah:  0.28092408662175083



In [44]:
calculated_features[calculated_features['label'].eq('control') & mask][['SID', feature]]

,SID,FDs_20SizeBy1_ArtVein__Fractal_Dimension
8,62,1.983194
17,63,2.040068
26,64,1.929826
35,68,2.024104
44,70,2.065196
53,71,1.953676
62,72,1.910128
71,82,2.000261
80,87,2.022505
89,98,2.004986
